<a href="https://colab.research.google.com/github/dasigiraghu96/Deeplearning_Computer_Vision/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
tf.test.gpu_device_name

<function tensorflow.python.framework.test_util.gpu_device_name>

In [ ]:
!nvidia-smi

Fri Mar 12 09:23:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config=ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=0.5
config.gpu_options.allow_growth=True
session=InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
Image_size=[224,224]
train_path='/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
valid_path='/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

In [ ]:
#VGG16 data preprocessing
#input_shape=rgb channel add [3]
#take weights from imagenet competetion
#include_top=False will remove flatten,fully connected layer,output layer used in imagenet competetion
#here we need only 2 classes
#input layer will also be skipped,by specifying input shape we are specifying our own input shape for our data
vgg16=VGG16(input_shape=Image_size+[3],weights='imagenet',include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
VGG16().summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [ ]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#don't train existing weights
for layer in vgg16.layers:
  layer.trainable=False 

In [ ]:
folders=glob('/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/train/*')
folders

['/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/train/dogs',
 '/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/train/cats']

In [ ]:
x=Flatten()(vgg16.output)

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)

In [ ]:
model=Model(inputs=vgg16.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2000 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1000 images belonging to 2 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
63/63 [==============================] - 1616s 25s/step - loss: 0.9450 - accuracy: 0.6638 - val_loss: 0.2395 - val_accuracy: 0.8930
Epoch 2/50
63/63 [==============================] - 41s 654ms/step - loss: 0.2377 - accuracy: 0.9045 - val_loss: 0.2327 - val_accuracy: 0.9000
Epoch 3/50
63/63 [==============================] - 41s 654ms/step - loss: 0.1797 - accuracy: 0.9334 - val_loss: 0.2658 - val_accuracy: 0.8910
Epoch 4/50
63/63 [==============================] - 41s 651ms/step - loss: 0.1549 - accuracy: 0.9426 - val_loss: 0.1880 - val_accuracy: 0.9200
Epoch 5/50
63/63 [==============================] - 41s 649ms/step - loss: 0.1160 - accuracy: 0.9543 - val_loss: 0.2069 - val_accuracy: 0.9150
Epoch 6/50
63/63 [==============================] - 41s 651ms/step - loss: 0.1025 - accuracy: 0.9679 - val_loss: 0.1882 - val_accuracy: 0.9230
Epoch 7/50
63/63 [==============================] - 42s 661ms/step - loss: 0.1043 - accuracy: 0.9591 - val_loss: 0.1975 - val_accuracy: 0.9220

In [ ]:
from keras.models import load_model
model.save('VGG16.h5')
model=load_model('VGG16.h5')


In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/cats_and_dogs_filtered/cats_and_dogs_filtered/validation/dogs/dog.2002.jpg', 
                            target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
a=np.argmax(model.predict(result), axis=1) #returns indices of maximum values
if a==0:
  print('The image is classified as Cat')
else:
  print('The image is classified as dog')